Global setup

In [52]:
n=2
k=2
r=2
q=41 #Lithium AAA?  #same as Alkaline D
#q should be divisible by 8 for NTT
η=1 #secret key range, prefered 2 in Alkaline
γ=2^4 #y coeff range, power of 2, exponent about 2/3 of q?  Earlier version was *not* power of 2
R.<x> = ZZ[]

In [2]:
q

41

In [3]:
is_prime(q)

True

In [4]:
qlen = ceil(log(q,2))
qlen

6

In [5]:
τ = 1 #number of 1's in c
ln(binomial(4, τ)).n()+τ #challenge entropy, should be between n/2 and n

2.38629436111989

In [6]:
β = τ*η
β

1

Chance of $\mathbf{y}_i$-coefficient passing:

In [8]:
((2*(γ-β)-1)/(2*γ-1))^(n*k).n()

0.765852644390328

Expected number of repetitions:

In [9]:
1/(((2*(γ-β)-1)/(2*γ-1))^(n*k))^2.n()

1.70494183535004

Alice generates a signing key

In [10]:
from pprint import pprint
A = matrix([[randrange(0,q)*x+randrange(0,q), randrange(0,q)*x+randrange(0,q)], 
            [randrange(0,q)*x+randrange(0,q), randrange(0,q)*x+randrange(0,q)]])
A

[      21     19*x]
[3*x + 18       39]

In [33]:
s1 = matrix([[randrange(-η,η+1)*x+randrange(-η,η+1), randrange(-η,η+1)*x+randrange(-η,η+1),], 
            [randrange(-η,η+1)*x+randrange(-η,η+1), randrange(-η,η+1)*x+randrange(-η,η+1)]])
(s1)

[-x + 1      x]
[-x - 1  x - 1]

In [34]:
s2 = matrix([[randrange(-η,η+1)*x+randrange(-η,η+1), randrange(-η,η+1)*x+randrange(-η,η+1),], 
            [randrange(-η,η+1)*x+randrange(-η,η+1), randrange(-η,η+1)*x+randrange(-η,η+1)]])
(s1)

[-x + 1      x]
[-x - 1  x - 1]

$s_1$ and $s_2$ are private

In [35]:
t = matrix(((A*s1 + s2) % (x^n+1) % q))
t

[        0    x - 20]
[29*x - 18 15*x + 39]

$A$ and $t$ are public

Alice encodes her message

In [36]:
M = 'HOLA'
M_list = [Mod(ord(ch) - ord("A") + 1,q).lift_centered() for ch in M]
M_list

[8, 15, 12, 1]

Pick nonces and generate $w$

In [37]:
y1 = matrix([[randrange(-(γ-1), γ)*x+randrange(-(γ-1), γ)], 
            [randrange(-(γ-1), γ)*x+randrange(-(γ-1), γ)]])
y1

[12*x - 2]
[2*x + 14]

In [38]:
y2 = matrix([[randrange(-(γ-1), γ)*x+randrange(-(γ-1), γ)], 
            [randrange(-(γ-1), γ)*x+randrange(-(γ-1), γ)]])
y2

[ -8*x - 12]
[-10*x - 11]

In [39]:
w = matrix(((A*y1+y2) % (x^n+1) % q)).apply_map(lambda f: f.map_coefficients(lambda r: Mod(r,q).lift_centered() ))
w

[18*x - 10]
[-9*x + 12]

Start the "non-cryptographic hash/XOF (combined?)"

In [40]:
w_list = [c for row in w for p in row for c in p.padded_list(n)]  #flatten the matrix down to coefficients
w_list

[-10, 18, 12, -9]

Diffusion box (D-box):

In [41]:
dboxlen = qlen

In [42]:
one = vector([1]*n*k)
(2*vector(M_list)+one).dot_product(2*vector(w_list)+one)

1398

In [43]:
def Dboxraw(list1, list2, itemlen):
    one = vector([1]*len(list1))
    newitem = (2*vector(list1)+one).dot_product(2*vector(list2)+one)  # make sure all bits of input are used
    out = (newitem//2) % (2**itemlen)
    return out

In [44]:
Dboxraw(M_list, w_list, dboxlen)

59

In [45]:
def Dbox(list1, list2, itemlen):
    one = vector([1]*len(list1))
    newitem = (2*vector(list1)+one).dot_product(2*vector(list2)+one)  # make sure all bits of input are used
    out = (newitem//2) % (2**itemlen)
    out = format(out, f'0{itemlen}b')
    return out

In [46]:
c_hash = Dbox(M_list, w_list, 6)
c_hash

'111011'

Use D-box output to sample a sparse polynomial

In [47]:
clenbits = log(n,2)
clenbits

1

In [53]:
def sample_in_ball(digest):  # digest should be at least ****
    c = [0] * n * r
    k = τ
    for i in range(n-τ, n):
        while int(digest[k:k+clenbits], 2) > i:
            k = k+clenbits
        j = int(digest[k:k+clenbits], 2)
        c[i] = c[j]
        c[j] = (-1)^int(digest[i+τ-n], 2)
        k = k+clenbits
    return c

Generate the signature


In [61]:
sample_in_ball(c_hash)

[0, -1, 0, 0]

In [75]:
c = matrix([[R(sample_in_ball(c_hash)[0:2])], [R(sample_in_ball(c_hash)[2:4])]])
c

[-x]
[ 0]

In [76]:
z1 = matrix((y1 + s1*c) % (x^n+1))
(z1)

[11*x - 3]
[3*x + 13]

In [77]:
z2 = matrix((y2 + s2*c) % (x^n+1))
(z2)

[ -9*x - 13]
[-10*x - 10]

Is $\|\mathbf{z}_1\|_\infty$ or $\|\mathbf{z}_2
\|_\infty \geq \gamma-\beta$?

In [79]:
γ-β

15

If so, reject and pick new $\mathbf{y}_i$

If not, $(\mathbf{z}_1, \mathbf{z}_2, c)$ is the signature

In [80]:
(z1, z2, c)

(
[11*x - 3]  [ -9*x - 13]  [-x]
[3*x + 13], [-10*x - 10], [ 0]
)

Bob verifies the signature

In [82]:
wprime = matrix((A*z1+z2-t*c) % (x^n+1) % q).apply_map(lambda f: f.map_coefficients(lambda r: Mod(r,q).lift_centered() ))
wprime

[18*x - 10]
[-9*x + 12]

Is $\|\mathbf{z}_i\|_\infty < \gamma-\beta$?

In [83]:
z1

[11*x - 3]
[3*x + 13]

In [84]:
z2

[ -9*x - 13]
[-10*x - 10]

In [85]:
γ-β

15

Yes

Is $c = H(M , w')$?

In [86]:
wprime_list = [c for row in wprime for p in row for c in p.padded_list(n)]  #flatten the matrix down to coefficients
wprime_list

[-10, 18, 12, -9]

In [87]:
cprime_hash = Dbox(M_list, wprime_list, dboxlen)
cprime_hash

'111011'

In [90]:
cprime = matrix([[R(sample_in_ball(c_hash)[0:2])], [R(sample_in_ball(c_hash)[2:4])]])
cprime

[-x]
[ 0]

In [91]:
c == cprime

True

Yes.  The signature is accepted.